In [1]:
import numpy as np
from aocd.models import Puzzle

# Tips

## Define the object
- `puzzle = Puzzle(year=2017, day=20)` 
- `Puzzle(2017, 20) at 0x107322978 - Particle Swarm>`
- get the input `puzzle.input_data`
- subbit by setting:
  - `puzzle.answer_a = value_a`
  - `puzzle.answer_b = value_b`

## Transform to list variables on multiple lines: 
 - t = '''asd
        asd 
        asd 
        asd'''.split('\n')

## Map a string list to integer 
- `map(int, list)`
- `np.array(p.input_data.split('\n'), dtype='int')`

## Day 1

In [ ]:
puzzle = Puzzle(year=2024, day=1)


def get_distance(a: np.array, b: np.array) -> int:
    a.sort()
    b.sort()
    return np.sum(np.abs(a - b))


def get_similarity(a: np.array, b: np.array) -> int:
    sim = 0
    for el in a:
        sim += el * np.sum(b == el)
    return sim


# tests
data = puzzle.examples[0].input_data.split("\n")
data = np.array([list(map(int, line.split())) for line in data])
dist = get_distance(data[:, 0], data[:, 1])
assert dist == 11

sim = get_similarity(data[:, 0], data[:, 1])
assert sim == 31

# part a
data = puzzle.input_data.split("\n")
data = np.array([list(map(int, line.split())) for line in data])
dist = get_distance(data[:, 0], data[:, 1])
puzzle.answer_a = dist

# part b
sim = get_similarity(data[:, 0], data[:, 1])
puzzle.answer_b = sim

coerced int64 value np.int64(1580061) for 2024/01 to '1580061'
coerced int64 value np.int64(23046913) for 2024/01 to '23046913'


# Day 2

In [48]:
puzzle = Puzzle(year=2024, day=2)

In [49]:
puzzle

got 404 status code
Please don't repeatedly request this endpoint before it unlocks! The calendar countdown is synchronized with the server time; the link will be enabled on the calendar the instant this puzzle becomes available.



AocdError: HTTP 404 at https://adventofcode.com/2024/day/2

# Day 3

# Day 4

# Day 5

# Day 6

# Day 7

# Day 8

# Day 9

# Day 10

# Day 11

# Day 12

# Day 13

# Day 14

# Day 15

# Day 16

# Day 17

# Day 18

# Day 19

# Day 20

# Day 21

# Day 22

# Day 23

# Day 24

# Day 25